In [12]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers
from langchain.vectorstores import Chroma
from uuid import uuid4


In [2]:
#Extract data from the PDF
def load_pdf(data):
    loader = DirectoryLoader(data,
                    glob="*.pdf",
                    loader_cls=PyPDFLoader)
    
    documents = loader.load()

    return documents

In [20]:
extracted_data = load_pdf("D:\Generative Ai\MediQuery-LLAMA2-LANGCHAIN-RAG\data")

In [7]:
# extracted_data
extracted_data

In [21]:
#Create text chunks
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 20)
    text_chunks = text_splitter.split_documents(extracted_data)

    return text_chunks

In [42]:
text_chunks = text_split(extracted_data)
print("length of my chunk:", len(text_chunks))

length of my chunk: 5860


In [44]:
# text_chunks
text_chunks

[Document(page_content='The GALE\nENCYCLOPEDIA\nof MEDICINE\nSECOND EDITION', metadata={'source': 'D:\\Generative Ai\\MediQuery-LLAMA2-LANGCHAIN-RAG\\data\\Medical_book.pdf', 'page': 1}),
 Document(page_content='The GALE\nENCYCLOPEDIA\nof MEDICINE\nSECOND EDITION\nJACQUELINE L. LONGE, EDITOR\nDEIRDRE S. BLANCHFIELD, ASSOCIATE EDITOR\nVOLUME\nA-B\n1', metadata={'source': 'D:\\Generative Ai\\MediQuery-LLAMA2-LANGCHAIN-RAG\\data\\Medical_book.pdf', 'page': 2}),
 Document(page_content='STAFF\nJacqueline L. Longe, Project Editor\nDeirdre S. Blanchfield, Associate Editor\nChristine B. Jeryan, Managing Editor\nDonna Olendorf, Senior Editor\nStacey Blachford, Associate Editor\nKate Kretschmann, Melissa C. McDade, Ryan\nThomason, Assistant Editors\nMark Springer, Technical Specialist\nAndrea Lopeman, Programmer/Analyst\nBarbara J. Yarrow,Manager, Imaging and Multimedia\nContent\nRobyn V . Young,Project Manager, Imaging and\nMultimedia Content\nDean Dauphinais, Senior Editor, Imaging and', metad

In [13]:
#download embedding model
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [14]:
embeddings = download_hugging_face_embeddings()

In [5]:
embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [6]:
query_result = embeddings.embed_query("Hello world")
print("Length", len(query_result))

Length 384


In [17]:
# query_result

In [15]:
from langchain_chroma import Chroma

vector_store = Chroma(
    collection_name="mediquery",
    embedding_function=embeddings,
    persist_directory="../chroma_langchain_db",  # Where to save data locally, remove if not necessary
)

In [47]:
uuids = [str(uuid4()) for _ in range(len(text_chunks))]

vector_store.add_documents(documents=text_chunks, ids=uuids)

['e3035da5-3444-4b5b-828f-cf8936a1e68e',
 '93de7012-c930-41d2-9631-ea7eb7e06eb1',
 'cc2585dd-a3d2-44c5-a9cd-d422b426592e',
 'ccfeedbe-61bb-4a0a-8a09-b7ac9853cdbb',
 '5dfa9582-806e-4896-bbe2-559aee2d247c',
 '061c8d95-d0bd-4a92-a879-90516db7c7a2',
 'b0d235e7-7e5d-432e-883e-b0692a5be1bb',
 '06f8e3bf-d3d1-4f11-aa4e-a6711b487f56',
 '7a5e2384-b54f-41b9-92fe-faf7fb457657',
 '504c39a5-e215-415e-81b9-6a3035807b93',
 '3ba03d31-f933-4648-8a50-8bcf75b8d850',
 'bf4571b8-1390-4080-a7e9-9caf2f5bb31b',
 '8332a54a-67d3-464c-a14a-68ef00e6ecc5',
 '4d951d51-02a0-4134-ad75-25f4b55aa2cb',
 '47b53c3c-35e8-4374-a1fd-5a14cf53e25e',
 '5c499782-f379-429f-87c9-34d2f7feeeac',
 '65d63d0f-9946-4e3e-b178-64565267cda7',
 'eb49a2bb-b8b5-4251-a911-c8ce99963c5e',
 '1a99b000-000a-4c42-856a-a336bce35799',
 '8e22f851-dddf-4a83-8a45-8d2bff71fedd',
 '2ea61b9a-607b-4201-8cd3-b17475cdea6a',
 '05b7e78b-24d8-4c1d-82fa-5b3f929c4e3a',
 '86876939-6cf7-4669-82bc-aac1ec2553c5',
 'bd9b4484-935d-46e7-91b5-d938c52ff050',
 '8be69704-261c-

In [6]:
results = vector_store.similarity_search_by_vector(
    embedding=embeddings.embed_query("what is acne"), k=2
)
for doc in results:
    print(f"* {doc.page_content} [{doc.metadata}]")

* GALE ENCYCLOPEDIA OF MEDICINE 226
Acne
GEM - 0001 to 0432 - A  10/22/03 1:41 PM  Page 26 [{'page': 39, 'source': 'D:\\Generative Ai\\MediQuery-LLAMA2-LANGCHAIN-RAG\\data\\Medical_book.pdf'}]
* Acidosis see Respiratory acidosis; Renal
tubular acidosis; Metabolic acidosis
Acne
Definition
Acne is a common skin disease characterized by
pimples on the face, chest, and back. It occurs when the
pores of the skin become clogged with oil, dead skin
cells, and bacteria.
Description
Acne vulgaris, the medical term for common acne, is
the most common skin disease. It affects nearly 17 million
people in the United States. While acne can arise at any [{'page': 37, 'source': 'D:\\Generative Ai\\MediQuery-LLAMA2-LANGCHAIN-RAG\\data\\Medical_book.pdf'}]


In [7]:
prompt_template="""
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

In [16]:
PROMPT=PromptTemplate(template=prompt_template, input_variables=["context", "question"])
chain_type_kwargs={"prompt": PROMPT}

In [17]:
llm=CTransformers(model="D:\Generative Ai\MediQuery-LLAMA2-LANGCHAIN-RAG\LLM/llama-2-7b-chat.ggmlv3.q4_0.bin",
                  model_type="llama",
                  config={'max_new_tokens':512,
                          'temperature':0.8})

In [18]:
# Create the retriever
retriever = vector_store.as_retriever(search_kwargs={'k': 2})

In [19]:
print(retriever)
print(type(retriever))

tags=['Chroma', 'HuggingFaceEmbeddings'] vectorstore=<langchain_chroma.vectorstores.Chroma object at 0x000001F8834CB6D0> search_kwargs={'k': 2}
<class 'langchain_core.vectorstores.base.VectorStoreRetriever'>


In [20]:
qa = RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=retriever,
    return_source_documents=True,
    chain_type_kwargs=chain_type_kwargs
)


In [ ]:
while True:
    user_input=input(f"Input Prompt:")
    result=qa({"query": user_input})
    print("Response : ", result["result"])